# Cross-spectra visualisation (only one pair of cross-spectra)

In [1]:
import glob
import datatree
import xarray as xr
import numpy as np

# Lulu's scripts using xsarslc functions
import sys 
path_tools = '/home1/datahome/ljessel/'   # path to the toolkit directory
sys.path.append(path_tools)
from python_scripts.one_xspectra_plot import plot_one_xspec

## 0) Loading file

In [2]:
path = '/home/datawork-cersat-public/cache/project/sarwave/data/products/tests2/slc/iw/l1b/2023/330/'
safe = 'S1A_IW_XSP__1SDV_20231126T151943_20231126T152010_051390_063392_00FA_A02.SAFE'
fns = glob.glob(path+safe+'/*iw3*vv*.nc')  # the iw2 sub_swaths have a five 5 tile_sample whereas the 2 others (iw1 et iw3) have a 4 tile_sample
l1b_path = fns[0]
#print(l1b_path)

# Open the nc_file using xarray
l1b_ds = xr.open_mfdataset(l1b_path, group='intraburst')
display(l1b_ds)

<xarray.Dataset> Size: 6MB
Dimensions:                   (bt_thresh: 5, tile_line: 72, tile_sample: 38,
                               freq_sample: 506, freq_line: 51, c_sample: 2,
                               c_line: 2, k_gp: 4, phi_hf: 5, 2tau: 1,
                               lambda_range_max_macs: 10)
Coordinates:
  * bt_thresh                 (bt_thresh) int16 10B 5 50 100 150 200
    pol                       <U2 8B ...
    k_rg                      (tile_line, tile_sample, freq_sample) float32 6MB dask.array<chunksize=(72, 38, 506), meta=np.ndarray>
    k_az                      (freq_line) float32 204B dask.array<chunksize=(51,), meta=np.ndarray>
    line                      (tile_line) float32 288B dask.array<chunksize=(72,), meta=np.ndarray>
    sample                    (tile_line, tile_sample) float32 11kB dask.array<chunksize=(72, 38), meta=np.ndarray>
    longitude                 (tile_line, tile_sample) float32 11kB dask.array<chunksize=(72, 38), meta=np.ndarray>
    latitude                  (tile_line, tile_sample) float32 11kB dask.array<chunksize=(72, 38), meta=np.ndarray>
  * k_gp                      (k_gp) int16 8B 1 2 3 4
  * phi_hf                    (phi_hf) int16 10B 1 2 3 4 5
  * lambda_range_max_macs     (lambda_range_max_macs) float32 40B 50.0 ... 275.0
Dimensions without coordinates: tile_line, tile_sample, freq_sample, freq_line,
                                c_sample, c_line, 2tau
Data variables: (12/23)
    burst                     (tile_line) int16 144B dask.array<chunksize=(72,), meta=np.ndarray>
    incidence                 (tile_line, tile_sample) float32 11kB dask.array<chunksize=(72, 38), meta=np.ndarray>
    ground_heading            (tile_line, tile_sample) float32 11kB dask.array<chunksize=(72, 38), meta=np.ndarray>
    sensing_time              (tile_line, tile_sample) datetime64[ns] 22kB dask.array<chunksize=(72, 38), meta=np.ndarray>
    sigma0                    (tile_line, tile_sample) float32 11kB dask.array<chunksize=(72, 38), meta=np.ndarray>
    nesz                      (tile_line, tile_sample) float32 11kB dask.array<chunksize=(72, 38), meta=np.ndarray>
    ...                        ...
    land_flag                 (tile_line, tile_sample) bool 3kB dask.array<chunksize=(72, 38), meta=np.ndarray>
    burst_corner_longitude    (tile_line, c_sample, c_line) float32 1kB dask.array<chunksize=(72, 2, 2), meta=np.ndarray>
    burst_corner_latitude     (tile_line, c_sample, c_line) float32 1kB dask.array<chunksize=(72, 2, 2), meta=np.ndarray>
    cwave_params              (tile_line, tile_sample, k_gp, phi_hf, 2tau) float32 219kB dask.array<chunksize=(72, 38, 4, 5, 1), meta=np.ndarray>
    macs_Re                   (tile_line, tile_sample, lambda_range_max_macs, 2tau) float32 109kB dask.array<chunksize=(72, 38, 10, 1), meta=np.ndarray>
    macs_Im                   (tile_line, tile_sample, lambda_range_max_macs, 2tau) float32 109kB dask.array<chunksize=(72, 38, 10, 1), meta=np.ndarray>
Attributes: (12/24)
    orbit_pass:             Ascending
    azimuth_time_interval:  0.002055556299999998
    short_name:             SENTINEL1_DS:S1A_IW_SLC__1SDV_20231126T151943_202...
    product:                SLC
    swath:                  IW
    platform_heading:       -13.94889495121479
    ...                     ...
    name:                   SENTINEL1_DS:/home/datawork-cersat-public/project...
    tile_overlap_line:      3000
    tile_width_sample:      5000
    azimuth_steering_rate:  1.397440818
    tile_overlap_sample:    3000
    ipf:                    3.71

## 1) Plotting xspec

<div class="alert alert-block alert-warning">
<b>Important note:</b> To be able to plot cross-sepctra, the netCDFs should contain the xspectra_2tau_Re and the xspectra_2tau_Im variables
</div>

<div class="alert alert-block alert-info">
<b>Note :</b> To have an overview of the mean lon/lat position of the tiles, we represent a [longitude, latitude] array for each one of them.<br>
    The "first" tile is located on the upper-right corner of the sub-swath & the "last" tile is located on the lower-left corner of the sub-swath.

</div>

In [9]:
lon = l1b_ds.longitude.values
lat = l1b_ds.latitude.values
lonlat_bytile = np.empty(lon.shape + (2,), dtype=float)
lonlat_bytile[:, :, 0] = lon
lonlat_bytile[:, :, 1] = lat
display(lonlat_bytile, lonlat_bytile.shape) # the tile_line corresponds to the first lonlat_bytile dimension & the tile_sample corresponds to the second lonlat_bytile dimension  

array([[[123.99074554, -11.80281734],
        [123.97284698, -11.79874706],
        [123.95497131, -11.79467869],
        ...,
        [123.27568054, -11.63919735],
        [123.25780487, -11.63508224],
        [123.23993683, -11.63096809]],

       [[123.98665619, -11.82054043],
        [123.96875763, -11.81646919],
        [123.95088196, -11.81240082],
        ...,
        [123.27153778, -11.65689754],
        [123.25366211, -11.65278149],
        [123.23579407, -11.64866638]],

       [[123.9825592 , -11.8382616 ],
        [123.96466064, -11.83418941],
        [123.94678497, -11.83012009],
        ...,
        [123.26739502, -11.67459488],
        [123.24951172, -11.67047882],
        [123.23164368, -11.66636276]],

       ...,

       [[123.66140747, -13.21681595],
        [123.64344025, -13.21270847],
        [123.62545776, -13.20859623],
        ...,
        [122.94283295, -13.05150223],
        [122.92485809, -13.04733849],
        [122.90692139, -13.04318237]],

       [[123.65

(72, 43, 2)

In [10]:
plot_one_xspec(l1b_path, tile_line=2, tile_sample=2, output_dir='/home1/datahome/ljessel/Plots/cross-spectras/')

KeyError: 'Could not find node at xspectra_2tau_Re'